# Tracking

This page discusses the MLFlow component responsible for tracking - monitoring of the various attempts to train a model.

In [ ]:
import mlflow
import numpy as np

import mlflow.models
import mlflow.sklearn
from mlflow.tracking import MlflowClient

from sklearn.linear_model import LinearRegression


client = MlflowClient()
mlflow.set_tracking_uri("http://localhost:5000/")

## Experiment

An experiment is a set of runs with different parameters for the same model. There are a few functions to manage the runs with very transparent names:

- `mlflow.create_experiment`.
- `mlflow.set_experiment`.

---

The following cell sets the "some experiment".

In [2]:
mlflow.set_experiment("some experiment")

2025/08/08 14:48:45 INFO mlflow.tracking.fluent: Experiment with name 'some experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/155825779609388391', creation_time=1754653725602, experiment_id='155825779609388391', last_update_time=1754653725602, lifecycle_stage='active', name='some experiment', tags={}>

Based on the log message, you can see that "some experiment" was created during the `set_experiment` call.

## Run

Run in mlflow refers to the execution of a machine learning experiment or a piece of code that you want to track and record. It represents a specific execution instance with associated metadata and recorded metrics, parameters, artifacts, and tags.

---

Run can be started with `mlflow.start_run()` and end it with `mlflow.end_run()`. By passing the `run_name` parameter to the `mlflow.start_run()` function, you can define the name of the run. The following cell demonstrates how it may be:

In [10]:
import mlflow

mlflow.start_run(run_name="my_run_name")
mlflow.end_run()

🏃 View run my_run_name at: http://localhost:5000/#/experiments/0/runs/925288ded96849d38de4ab46b5f9e3e4
🧪 View experiment at: http://localhost:5000/#/experiments/0


**Note** it's OK to have several runs with the same name - you can think of them as different versions of the same run.

The most popular approach to start a run is using `mlflow.start_run` as a context manager expression. The run begins upon entering the context and ends upon exiting it.

The follwoing cell shows launching an empty run using exactly context manager approach.

In [18]:
with mlflow.start_run(run_name="with run"):
    pass

🏃 View run with run at: http://localhost:5000/#/experiments/101046082978810346/runs/b96f254b78f14e7ca8e19b2ec23146f9
🧪 View experiment at: http://localhost:5000/#/experiments/101046082978810346


Use the `mlflow.search_runs` command to list all available runs.

In [17]:
mlflow.search_runs(search_all_experiments=True)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.user
0,7ac6c477dc724ea5ad07065cdae6794e,101046082978810346,FINISHED,mlflow-artifacts:/101046082978810346/7ac6c477d...,2025-08-08 09:12:47.469000+00:00,2025-08-08 09:12:47.480000+00:00,/home/fedor/.virtualenvironments/python/lib/py...,LOCAL,with run,fedor
1,925288ded96849d38de4ab46b5f9e3e4,0,FINISHED,mlflow-artifacts:/0/925288ded96849d38de4ab46b5...,2025-08-08 08:46:34.027000+00:00,2025-08-08 08:46:34.041000+00:00,/home/fedor/.virtualenvironments/python/lib/py...,LOCAL,my_run_name,fedor
2,c243c89b84fc4c62ad756d4512d6f4ee,0,FINISHED,mlflow-artifacts:/0/c243c89b84fc4c62ad756d4512...,2025-08-08 08:41:22.880000+00:00,2025-08-08 08:44:31.008000+00:00,/home/fedor/.virtualenvironments/python/lib/py...,LOCAL,my_run_name,fedor


## Log

The following table represents things you can log with mlflow and their corresponding functions.

| **Loggable Entity**         | **MLflow Function(s)**                                                                                                                                |
| --------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------- |
| Parameters (single)         | `mlflow.log_param(key, value)`                                                                                                                        |
| Parameters (multiple)       | `mlflow.log_params(dict_of_params)`                                                                                                                   |
| Metrics (single)            | `mlflow.log_metric(key, value, step=None)`                                                                                                            |
| Metrics (multiple)          | Metric logging is usually done one at a time — there's no built-in `log_metrics()`, but you can loop over a dict to call `log_metric` multiple times. |
| Artifacts (single file)     | `mlflow.log_artifact(local_path, artifact_path=None)`                                                                                                 |
| Artifacts (directory)       | `mlflow.log_artifacts(local_dir, artifact_path=None)`                                                                                                 |
| Models (generic)            | `mlflow.log_model(model, artifact_path, **kwargs)`                                                                                                    |
| Models (framework-specific) | - `mlflow.sklearn.log_model(...)`<br>- `mlflow.pytorch.log_model(...)`<br>- `mlflow.tensorflow.log_model(...)`, etc.                                  |

---

The following code represents the procedure of model logging.

In [8]:
model = LinearRegression().fit(
    X=np.random.normal(0, 2, size=(100, 2)),
    y=np.random.normal(0, 20, size=100)
)
signature = mlflow.models.infer_signature(
    np.random.normal(0, 2, size=(100, 2)),
    np.random.normal(0, 20, size=100)
)

with mlflow.start_run(run_name="test_run"):
    mlflow.sklearn.log_model(model, signature=signature)

🏃 View run test_run at: http://localhost:5000/#/experiments/155825779609388391/runs/57a4997e6e7d4b37b4237f1254ffd2f7
🧪 View experiment at: http://localhost:5000/#/experiments/155825779609388391
